In [1]:
import satpy
from satpy import Scene
from glob import glob
from satpy.writers import get_enhanced_image
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
from pyresample import create_area_def
from pathlib import Path
import xarray as xr
import yaml

### Process MODIS 1b data using satpy

In [4]:
# specify folder containing set of MODIS hdf files

folder = "AquaHkmLabSea2022b"
#folder = 'AquaQkmLabSeaOctNov22'
#folder = "Aqua1km"
file_loc = "/home/eefjg/OneDrive/Leeds/PhD/Data/MODIS/" + folder

filenames = sorted(glob(file_loc + "/*.hdf"))
geoloc_files = sorted(glob(file_loc + "/geoloc/*.hdf")) # folder for MYD03 geolocation files


- need to load geolocation files simultaneously
- maybe need a list of times/dates akin to scene_ids, with the two nc files listed under each.

### Checks and tests

In [10]:
# check available channels and composites

scn = Scene(reader='modis_l1b', filenames=[filenames[0],geoloc_files[0]])
print(scn.available_dataset_names())
print(scn.available_composite_names())


['1', '2', '3', '4', '5', '6', '7', 'latitude', 'longitude', 'satellite_azimuth_angle', 'satellite_zenith_angle', 'solar_azimuth_angle', 'solar_zenith_angle']
['natural_color', 'natural_color_raw', 'ocean_color', 'true_color', 'true_color_crefl', 'true_color_uncorrected']


In [11]:
# need to load the datasets we want before resampling
scn.load(['true_color', 'true_color_crefl', 'true_color_uncorrected'])

The following datasets were not created and may require resampling to be generated: DataID(name='true_color_uncorrected'), DataID(name='true_color_crefl'), DataID(name='true_color')


In [16]:
# resample to allow composite generation (e.g. solar_zenith_angle is at 250m res not 500m)
# use coarset area to downsample to 500m (dta resolution)

new_scn = scn.resample(scn.coarsest_area(), resampler='native')

In [13]:
# load composites from resampled scene
# new_scn.load(['true_color']) NB don't actually need to load all datasets again
new_scn['true_color']

<xarray.DataArray 'where-9e8df3cb4c71474f1f99581fb941624a' (bands: 3, y: 4060,
                                                            x: 2708)> Size: 132MB
dask.array<where, shape=(3, 4060, 2708), dtype=float32, chunksize=(1, 3080, 2708), chunktype=numpy.ndarray>
Coordinates:
    crs      object 8B +proj=longlat +ellps=WGS84 +type=crs
  * bands    (bands) <U1 12B 'R' 'G' 'B'
Dimensions without coordinates: y, x
Attributes: (12/18)
    reader:                  modis_l1b
    area:                    Shape: (4060, 2708)\nLons: <xarray.DataArray 'ge...
    ancillary_variables:     []
    sensor:                  modis
    resolution:              500
    coordinates:             ('longitude', 'latitude')
    ...                      ...
    wavelength:              None
    name:                    true_color
    _satpy_id:               DataID(name='true_color', resolution=500)
    prerequisites:           [DataQuery(name='1', modifiers=('sunz_corrected'...
    optional_prerequisites:  []
    mode:                    RGB

In [105]:
# tests for saving composites to file

new_scn.save_datasets(filename="{name}_{start_time:%Y%m%d_%H%M}.png", datasets = 'true_color_crefl', writer="simple_image")

/home/eefjg/miniconda3/envs/satpy/lib/python3.12/site-packages/satpy/modifiers/_crefl_utils.py:534: RuntimeWarning: invalid value encountered in log
  return np.exp(-np.exp(self._ah2o + self._bh2o * np.log(self._airmass * UH2O_MODIS)))


In [17]:
new_scn.save_datasets(filename="test.nc",writer="cf");

/home/eefjg/miniconda3/envs/satpy/lib/python3.12/site-packages/satpy/modifiers/_crefl_utils.py:534: RuntimeWarning: invalid value encountered in log
  return np.exp(-np.exp(self._ah2o + self._bh2o * np.log(self._airmass * UH2O_MODIS)))


In [18]:
ds = xr.open_dataset("test.nc")
ds

<xarray.Dataset> Size: 484MB
Dimensions:                 (bands: 3, y: 4060, x: 2708)
Coordinates:
  * bands                   (bands) <U1 12B 'R' 'G' 'B'
    longitude               (y, x) float32 44MB ...
    latitude                (y, x) float32 44MB ...
Dimensions without coordinates: y, x
Data variables:
    true_color              (bands, y, x) float32 132MB ...
    true_color_crefl        (bands, y, x) float32 132MB ...
    true_color_uncorrected  (bands, y, x) float32 132MB ...
Attributes:
    history:      Created by pytroll/satpy on 2024-07-31 15:30:16.892653
    Conventions:  CF-1.7

### Save slices as both png images and nc files

In [19]:
def save_slice(files, filepath, composite):
    '''saves a 1024 wide slice of the image to remove distorted areas (as Geiss 2024)
     Also saves nc file of rgb values for given composite'''
    # open file
    scn = Scene(reader='modis_l1b', filenames=files)
    scn.load([composite])
    
    # resample to allow composite generation 
    new_scn = scn.resample(scn.coarsest_area(), resampler='native')    
    
    # slice off the edges
    scn_slice = new_scn[:,:,843:1867] # composites have three dimensions where x is the third
    # flip the image
    scn_slice =  scn_slice[:,::-1,::-1]
    # save the sliced image
    scn_slice.save_datasets(filename=filepath+"/{name}_{start_time:%Y%m%d_%H%M}.png",writer="simple_image")
    # save an nc file
    scn_slice.save_datasets(filename=filepath+"/{name}_{start_time:%Y%m%d_%H%M}.nc",writer="cf");
    

In [5]:
# make filepath for image slices within data folder

sliced_loc = file_loc + "/sliced"
filepath = Path(sliced_loc)
filepath.mkdir(exist_ok=True, parents=True)

In [108]:
composite = 'true_color'

for filepair in zip(filenames, geoloc_files):
    files = list(filepair)
    save_slice(files, str(filepath), composite);

The following datasets were not created and may require resampling to be generated: DataID(name='true_color')
The following datasets were not created and may require resampling to be generated: DataID(name='true_color')
The following datasets were not created and may require resampling to be generated: DataID(name='true_color')
The following datasets were not created and may require resampling to be generated: DataID(name='true_color')
The following datasets were not created and may require resampling to be generated: DataID(name='true_color')
The following datasets were not created and may require resampling to be generated: DataID(name='true_color')
The following datasets were not created and may require resampling to be generated: DataID(name='true_color')
The following datasets were not created and may require resampling to be generated: DataID(name='true_color')
The following datasets were not created and may require resampling to be generated: DataID(name='true_color')
The follow